### Quantum Battery project

Step 1. Run below cell to import all package.

In [1]:
from qsee.compilation.qcompilation import QuantumCompilation
from qsee.compilation.qsp import QuantumStatePreparation
from qsee.core import ansatz, state, gradient, optimizer
from qsee.backend import constant, utilities
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import os
import base.Gradient
import base.circuit
import base.object
import numpy as np, qiskit
%load_ext autoreload
%autoreload 2

Step 2. Run below cell to make sure that your environment is ok or not

In [2]:
qsp = QuantumStatePreparation.prepare([1, 0, 0, 0])

Step 3. Coding!!!

In [ ]:
t = 10
num_qubits = 2
thetas = np.random.rand(2*(num_qubits-1))*2*np.pi
thetasmax,thetass=base.Gradient.gradient_descent(num_qubits,t,thetas)


In [ ]:
def test(num_qubits, t1,thetas,shots):
    """

        INPUT:
          t: time
          thetas: array parameter (length 2*(N-1))
          shots: number of measurement times

    """
    time=np.linspace(0,t1,shots)
    arrayW=[]
    arrayP=[]
    for t in time:
        w = base.object.E(num_qubits, t,thetas) - base.object.E(num_qubits, 0,thetas)
        if(t!=0):
            p = w/t
        else:
            p = 0
        arrayW.append(w)
        arrayP.append(p)
    return time,arrayP,arrayW
ltime,lP,lW=test(num_qubits,t,thetasmax)
plt.plot(ltime, lW, label="Work")
plt.plot(ltime, lP, label="Power")
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()